### Declan Worley
# Final Project - CRT Model




<img src="files/crt.png" height="100">

Cathode Ray Tubes (CRTs) are vacuum tubes that produce images by use of electron beams and a phospherescent screen. This is done by manipulation of the electron beams so that it strikes certain spots on the screen creating luminous spots. 

The source of the electron beams is the electron gun, which contains the hater, cathode, control grid, and anodes. The cathode is cylindrical and contains a layer of strontium and barium oxide which emits the electrons at the end of the tube. These electrons pass through the control grid and continue to pass through the anodes which help focus the beams.

Electrons eventually reach the deflecting plates which send the electron into certain areas of the fluorescent screen, which is used to create an image on the screen.

## Implementation

In our code, we will be modeling the manipulation of the electrons using the deflection plates.

The program will contain a target and let players try to hit the target with the electron by adjusting the magnetic field of the deflection plates.

To do this, we will first need to setup some vertical and horizon plates with a certain B for the magnetic fields, and an electron with a velocity, mass, and charge.


### Imports 

In [ ]:
# Importing
from vpython import *
import math
import random


### Objects
These are mainly objects used as targets in our program for purely visuals. They can be substituted for any simple object in VPython.

In [6]:
# Target object
def Target(pos=vec(0,0,0), color=vec(1,0,0)):
    #bound=sphere(pos=pos+vec(0,1,0),opacity=1/3, radius=5/4)
    body=cylinder(color=color,pos=pos, axis=vec(0,1,0), length=3/2)
    head=sphere(color=color,pos=body.pos+body.axis)
    eye=ellipsoid(color=vec(0,1,1), pos=head.pos+vec(0,0,1), width=1/4, height=1/3, length=1)
    lleg=ellipsoid(color=color, pos=body.pos+vec(-1/2,0,0), height=2, width=2/3, length=2/3)
    rleg=ellipsoid(color=color, pos=body.pos+vec(1/2,0,0), height=2, width=2/3, length=2/3)
    backpack=ellipsoid(color=color, pos=body.pos+body.axis/2+vec(0,0,-3/4), length=3/2, height=3/2, width=1)
    target=compound([body,head,eye,lleg,rleg,backpack], radius=5/4, c=color)

    return target

# Target Object (When its hit)
def Dead(pos=vec(0,0,0), color=vec(1,0,0)):
    body=cylinder(color=color,pos=pos+vec(0,-3/2,0), axis=vec(0,1,0), length=1)
    bbone=cylinder(color=vec(1,1,1),pos=body.pos+vec(0,2/3,0), axis=vec(0,1,0), radius=1/3)
    lbone=sphere(color=vec(1,1,1),pos=bbone.pos+bbone.axis+vec(-bbone.radius/2,0,0), axis=vec(0,1,0), radius=bbone.radius)
    rbone=sphere(color=vec(1,1,1),pos=bbone.pos+bbone.axis+vec(bbone.radius/2,0,0), axis=vec(0,1,0), radius=bbone.radius)
    lleg=ellipsoid(color=color, pos=body.pos+vec(-1/2,0,0), height=2, width=2/3, length=2/3)
    rleg=ellipsoid(color=color, pos=body.pos+vec(1/2,0,0), height=2, width=2/3, length=2/3)
    backpack=ellipsoid(color=color, pos=body.pos+body.axis/2+vec(0,0,-3/4), length=3/2, height=3/2, width=1)
    dead=compound([body,bbone,lbone,rbone,lleg,rleg,backpack], c=color)
    dead.axis=vec(0,1,0)

    return dead

# Knife (knife mode)
def Knife(pos=vec(0,0,0)):
    #bound=sphere(pos=pos, radius=0.2,opacity=1/3)
    handle =ellipsoid(color=vec(0,0,0),axis=vec(0,0,-1),pos=pos, length=2, width=1/2, height=2/3)
    guard = ellipsoid(color=vec(0,0,0),axis=vec(0,0,-1),pos=handle.pos+handle.axis/2, length=2/3, width=2/3, height=2)
    blade = ellipsoid(color=vec(1,1,1),axis=vec(0,0,-1), pos=handle.pos+handle.axis, length=3, width=1/4, height=1)
    knife=compound([handle, guard, blade], radius=5*2/3, q=1.6e-19)

    return knife



### Model
This is the main body of the code.

First we create the canvas for VPython, and then the objects like the target, plates, and electron, and a backdrop.

Next, we have our constants starting on line 30, that contains like all f our information about our electron, magnetic field magnitude, etc.

We then set up our magnetic fields from our plates, using a relation of between the power of magnetic field of each set of plates. We also create arrows here to visualize how big the magnetic field is from each plate.

After that, we create our loop in which we move our target, get keyboard input, calculate the force, momentum, velocity, and update the position, and also the collisions.


### Calculations

To move our electron we need a few calculations, which are used after line 112


Calculate initial momentum:

$\vec{p} = \dfrac{\vec{v}}{m}$

Calculate the magnetic force:

$\vec{F}_{mag} = q * ( \vec{B} \times \vec{v})$

Calculating new momentum:

$\vec{p}_{f} = \vec{p}_{i} + \vec{F}_{mag} * dt$

Calculating velocity from momentum

$\vec{v} = \dfrac{\vec{p}}{m}$

Updating the position:

$\vec{r}_{f} = \vec{r}_{i} + \vec{v} * dt$



In [ ]:
# Scene
scene = canvas(background=vec(0, 30, 10)/255, autoscale=False)

# Making target
b=Target(vec(0,0,-39), color=vec(1,1,0))

# Setting up the plates
topplate=box(pos=vec(0,10,0), width=10, length=10, height=1/16, axis=vec(1,0,0))
botplate=box(pos=vec(0,-10,0), width=10, length=10, height=1/16, axis=vec(1,0,0))
vertplates=compound([topplate,botplate])
leftplate=box(pos=vec(-10,0,0), width=10, length=10, height=1/16, axis=vec(0,1,0))
rightplate=box(pos=vec(10,0,0), width=10, length=10, height=1/16, axis=vec(0,1,0))
horzplates=compound([leftplate,rightplate])

#Making the electron
KNIFE = False
if(KNIFE == True):
    e=Knife(vec(0,0,topplate.width/1.5))
else:
    e=sphere(pos=vec(0,0,0), radius=0.5, color=color.blue )

# Backdrop
screen=box(pos=vec(0,0,-40), color=vec(0.2,0.2,0.2), height=40, width=40,length=1/16, axis=vec(0,0,1))

# Moving Camera
scene.camera.pos=e.pos+vec(0,0,15)
scene.camera.range=1
# scene.camera.forward=20

# Constants
emass = 1
q = 1
evel = vec(0,0,0)
theta = random.random() * 2 * math.pi
bvel = vec(math.cos(theta), math.sin(theta),0) * 3
vel = 10
fired = False
BMAX = emass*vel*screen.width/(2*q*(topplate.length)*abs(screen.pos.z)) # Scaling based on velocity given to electron, so that it is bounded in the backdrop
TP = 0.5
LP = 0.5

# Setting up Magnetic fields
BTOP = vec(0,-1,0)*TP*BMAX
BBOT = vec(0,1,0)*(1-TP)*BMAX
BLEFT = vec(1,0,0)*LP*BMAX
BRIGHT = vec(-1,0,0)*(1-TP)*BMAX
# Visuals for magnetic fields
tarrow=arrow(pos=topplate.pos, axis=(BTOP/BMAX)*10, color=vec(1,0,1),opacity=0.5)
barrow=arrow(pos=botplate.pos, axis=(BBOT/BMAX)*10, color=vec(1,0,1),opacity=0.5)
larrow=arrow(pos=leftplate.pos, axis=(BLEFT/BMAX)*10, color=vec(1,0,1),opacity=0.5)
rarrow=arrow(pos=rightplate.pos, axis=(BRIGHT/BMAX)*10, color=vec(1,0,1),opacity=0.5)


INC = 0.01
#Loop
dt = 1e-2 # Time interval
while True:
    rate(1/dt)

    #Moving target
    b.pos += bvel *dt
    if abs(b.pos.y) >= screen.height/2:
        bvel.y = -bvel.y
    if abs(b.pos.x) >= screen.width/2:
        bvel.x = -bvel.x

    #Resetting electron
    if(e.pos.z < screen.pos.z or abs(e.pos.y) > screen.height/2 or abs(e.pos.x) > screen.width/2):
        e.pos = vec(0,0,topplate.width/1.5)
        evel = vec(0,0,0)
        fired=False
        

    #Keyboard inputs
    k = keysdown() # a list of keys that are down
    if 'left' in k:
        if LP > 0:
            LP -= INC
            # print(LP)
    if 'right' in k:
        if LP < 1:
            LP += INC
            # print(LP)
    if 'down' in k:
        if TP < 1:
            TP += INC
            # print(TP)
    if 'up' in k:
        if TP > 0:
            TP -= INC
            # print(TP)
    if ' ' in k and not fired:
        # print("Fire")
        fired = True
        evel = vec(0,0,-1)*vel


    ep = evel*emass#electron momentum

    #Magnetic fields
    BTOP = vec(0,-1,0)*TP*BMAX
    BBOT = vec(0,1,0)*(1-TP)*BMAX
    BLEFT = vec(1,0,0)*LP*BMAX
    BRIGHT = vec(-1,0,0)*(1-LP)*BMAX
    tarrow.axis=(BTOP/BMAX)*10
    barrow.axis=(BBOT/BMAX)*10
    larrow.axis=(BLEFT/BMAX)*10
    rarrow.axis=(BRIGHT/BMAX)*10

    #Total Magnetic field
    BNET = BTOP+BBOT+BLEFT+BRIGHT

    # Magnetic force on electron
    FMag = vec(0,0,0) # If outside plates force = 0
    if abs(topplate.pos.z-e.pos.z) < (topplate.width/2): # If inside plates then force
        FMag = q*cross(evel, BNET)

    #Updating electron pos
    ep = ep+FMag*dt #calculating momentum
    evel = ep/emass #calculating velocity
    e.pos += evel*dt #position

    #Collisions with target
    if mag((e.pos+vec(0,0,-e.radius)-b.pos))<b.radius*2/3:
        b.visible=False
        a=b
        b=Dead(b.pos, b.c)
        del a
        break
    #print(e.pos)


<IPython.core.display.Javascript object>

## Future Plans
This model can be taken and improved in a lot of ways such as:
1. Scale values down to an actual electron (It was scaled up to work easier with VPython)

2. A scan system could be added in to further show how CRT monitors work

3. To continue with the game concept, you could add in more targets, change the movement of them, and even add in a Fmag vector to help visualize where the electron would go as a result of the plates since it can be confusing without knowing the right hand rule.


I think, given the time, a great improvement to this model would be to add in scanning, and display an image on the screen in the back as CRT TVs are able to.


## References
https://circuitglobe.com/cathode-ray-tube-crt.html

https://computer.howstuffworks.com/monitor7.htm

https://nationalmaglab.org/education/magnet-academy/watch-play/interactive/electromagnetic-deflection-in-a-cathode-ray-tube-ii